In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [134]:
K = 5
z_max = 2
z_values = np.linspace(-z_max, z_max, 2**K)
p_zeros = [0.15, 0.25, 0.5, 0.12, 0.32]
rhos = [0.1, 0.05, 0.08, 0.03, 0.08]
lgd = [1, 2, 3, 6, 2]
alpha = 0.50
num_samples = 10000

z_pdf_values = norm.pdf(z_values, loc=0, scale=1)
f_values = np.cumsum(z_pdf_values)
f_values /= f_values[-1]
f_inverse_values = [norm.ppf(value) for value in f_values]


In [95]:
def calculate_pkz(probability_default, rho, z):
    inside_bracket = ((np.interp(probability_default, z_values, f_inverse_values) - (np.sqrt(rho)) * z)/np.sqrt(1-rho))
    
    return np.interp(inside_bracket, z_values, f_values)

In [113]:
def compute_var_cvar(lambda_k,X_k_samples, confidence_level):
    expected_losses = np.dot(X_k_samples, lambda_k)  # Weighted sum of X_k(Z)

    # Compute empirical VaR
    alpha = 1 - confidence_level
    var_threshold = np.percentile(expected_losses, alpha * 100)

    # Compute empirical CVaR (Expected Shortfall)
    cvar = expected_losses[expected_losses <= var_threshold].mean()

    return var_threshold, cvar

In [91]:
def bool_default(pkz):
    compare_value = np.random.rand()
    return compare_value < pkz

In [140]:
def simulate():
    default_states = []
    for i in range(K):
        z_i = np.random.choice(z_values)
        pkz_i = calculate_pkz(p_zeros[i],rhos[i],z_i)
        default_states.append(bool_default(pkz=pkz_i))
        
    return default_states

In [143]:
simulations = []

for x in range(num_samples):
    simulations.append(simulate())


In [144]:
var, cvar = compute_var_cvar(lambda_k=lgd,X_k_samples=simulations,confidence_level=alpha)

print(var, cvar)

10.0 6.526114649681529
